In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("aggregated-occupation-level-data.csv",index_col=0)

In [3]:
levels_columns = [x for x in list(df.columns) if "_LVL" in x ]
importance_columns = [x for x in list(df.columns) if "_IMP" in x ]

In [4]:
#multiple the colums ie. TROUBLESHOOTING_LVL * TROUBLESHOOTING_IMPORTANCE
cols = []
for imp_col in importance_columns:
    prefix = "_".join(imp_col.split('_')[:-1])
    level_col = f'{prefix}_LVL'
    cols.append(prefix)
    if level_col in levels_columns:
        new_col_name = f'{prefix}'
        df[new_col_name] = df[imp_col] * df[level_col]

In [5]:
rankings = df[cols].sum(axis=1).sort_values(ascending=False).round(2).reset_index()
rankings.columns = ["OCCUPATION_2018_DESC","skill_diversity"]
exposure = df[['HUMAN_NUMERIC',
 'GPT4_NUMERIC',
 'GPT4_AUTOMATION_NUMERIC']].reset_index()
rankings = exposure.merge(rankings, on="OCCUPATION_2018_DESC").sort_values(by="skill_diversity",ascending=False)
rankings.to_csv("skill_diversity.csv")

In [6]:
rankings

,OCCUPATION_2018_DESC,HUMAN_NUMERIC,GPT4_NUMERIC,GPT4_AUTOMATION_NUMERIC,skill_diversity
0,Chief executives,0.357143,0.489796,0.418367,455.74
56,Bioengineers and biomedical engineers,0.550000,0.600000,0.416667,444.15
63,Mechanical engineers,0.506329,0.525316,0.363924,393.34
11,Industrial production managers,0.346154,0.410256,0.393162,389.98
57,Chemical engineers,0.346154,0.461538,0.403846,388.17
...,...,...,...,...,...
178,Maids and housekeeping cleaners,0.080000,0.000000,0.040000,113.05
164,Crossing guards and flaggers,0.250000,0.125000,0.125000,98.08
311,"Pressers, textile, garment, and related materials",0.017857,0.017857,0.017857,94.34
341,"Packers and packagers, hand",0.083333,0.083333,0.083333,90.04


In [14]:
exposed_tasks = pd.read_csv("../../input/gpts_labels/gpts_labels_new.csv")
skills = pd.read_csv("../../output/parsed_BLS_data/dwa.csv", index_col=0)
exposed_tasks = exposed_tasks.merge(skills, on=["Task"])
skill_activities = pd.read_csv("../../input/onet_2023/Skills to Work Activities.csv").iloc[:,[1,3]]
skill_activities.columns = ["skill","activity"]
exposed_tasks = exposed_tasks.merge(skill_activities, on="activity")
exposed_tasks

,O*NET-SOC Code,Title,Task ID,Task,human_labels,gpt_4_exposure,gpt_4_automation,human_numeric,gpt_4_numeric,gpt_4_automation_numeric,DWA ID,DWA Title,dwa_count,activity,skill
0,11-2022.00,Sales Managers,1,Resolve customer complaints regarding sales an...,E1,E2,T2,1.0,0.5,0.5,4.A.4.a.8.I03.D05,Resolve customer complaints or problems.,1.0,Performing for or Working Directly with the Pu...,Active Listening
1,11-2022.00,Sales Managers,1,Resolve customer complaints regarding sales an...,E1,E2,T2,1.0,0.5,0.5,4.A.4.a.8.I03.D05,Resolve customer complaints or problems.,1.0,Performing for or Working Directly with the Pu...,Speaking
2,11-2022.00,Sales Managers,1,Resolve customer complaints regarding sales an...,E1,E2,T2,1.0,0.5,0.5,4.A.4.a.8.I03.D05,Resolve customer complaints or problems.,1.0,Performing for or Working Directly with the Pu...,Social Perceptiveness
3,11-2022.00,Sales Managers,1,Resolve customer complaints regarding sales an...,E1,E2,T2,1.0,0.5,0.5,4.A.4.a.8.I03.D05,Resolve customer complaints or problems.,1.0,Performing for or Working Directly with the Pu...,Coordination
4,11-2022.00,Sales Managers,1,Resolve customer complaints regarding sales an...,E1,E2,T2,1.0,0.5,0.5,4.A.4.a.8.I03.D05,Resolve customer complaints or problems.,1.0,Performing for or Working Directly with the Pu...,Service Orientation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306964,53-3054.00,Taxi Drivers,23762,Drive taxicabs or privately owned vehicles to ...,E0,E0,T0,0.0,0.0,0.0,4.A.3.a.4.I01.D14,Drive passenger vehicles.,1.0,"Operating Vehicles, Mechanized Devices, or Equ...",Operation and Control
306965,53-3054.00,Taxi Drivers,23767,"Pick up passengers at prearranged locations, a...",E0,E0,T0,0.0,0.0,0.0,4.A.3.a.4.I01.D14,Drive passenger vehicles.,1.0,"Operating Vehicles, Mechanized Devices, or Equ...",Operations Monitoring
306966,53-3054.00,Taxi Drivers,23767,"Pick up passengers at prearranged locations, a...",E0,E0,T0,0.0,0.0,0.0,4.A.3.a.4.I01.D14,Drive passenger vehicles.,1.0,"Operating Vehicles, Mechanized Devices, or Equ...",Operation and Control
306967,53-6032.00,Aircraft Service Attendants,23848,"Tow aircraft to gates or hangars using tugs, t...",E0,E0,T0,0.0,0.0,0.0,4.A.3.a.4.I01.D10,Drive trucks or truck-mounted equipment.,1.0,"Operating Vehicles, Mechanized Devices, or Equ...",Operations Monitoring


In [15]:
all_tasks = pd.read_csv("../../input/onet_2023/Task Ratings.csv",index_col=0).reset_index()
all_tasks = all_tasks[all_tasks["Scale Name"] == "Importance"][["O*NET-SOC Code","Title","Task","Data Value"]].rename({"Data Value":"task_importance"},axis=1)
skills = pd.read_csv("../../output/parsed_BLS_data/dwa.csv", index_col=0)
all_tasks = all_tasks.merge(skills, on=["Task"])
skill_activities = pd.read_csv("../../input/onet_2023/Skills to Work Activities.csv").iloc[:,[1,3]]
skill_activities.columns = ["skill","activity"]
all_tasks = all_tasks.merge(skill_activities, on="activity")
all_tasks


,O*NET-SOC Code,Title,Task,task_importance,DWA ID,DWA Title,dwa_count,activity,skill
0,11-1011.00,Chief Executives,Appoint department heads or managers and assig...,4.48,4.A.4.b.4.I12.D39,"Direct organizational operations, projects, or...",0.5,"Guiding, Directing, and Motivating Subordinates",Active Listening
1,11-1011.00,Chief Executives,Appoint department heads or managers and assig...,4.48,4.A.4.b.4.I12.D39,"Direct organizational operations, projects, or...",0.5,"Guiding, Directing, and Motivating Subordinates",Writing
2,11-1011.00,Chief Executives,Appoint department heads or managers and assig...,4.48,4.A.4.b.4.I12.D39,"Direct organizational operations, projects, or...",0.5,"Guiding, Directing, and Motivating Subordinates",Speaking
3,11-1011.00,Chief Executives,Appoint department heads or managers and assig...,4.48,4.A.4.b.4.I12.D39,"Direct organizational operations, projects, or...",0.5,"Guiding, Directing, and Motivating Subordinates",Critical Thinking
4,11-1011.00,Chief Executives,Appoint department heads or managers and assig...,4.48,4.A.4.b.4.I12.D39,"Direct organizational operations, projects, or...",0.5,"Guiding, Directing, and Motivating Subordinates",Active Learning
...,...,...,...,...,...,...,...,...,...
295145,53-7073.00,Wellhead Pumpers,Drive trucks to transport high-pressure pumpin...,3.40,4.A.3.a.4.I01.D06,Operate vehicles or material-moving equipment.,1.0,"Operating Vehicles, Mechanized Devices, or Equ...",Operation and Control
295146,53-7081.00,Refuse and Recyclable Material Collectors,"Drive trucks, following established routes, th...",4.73,4.A.3.a.4.I01.D06,Operate vehicles or material-moving equipment.,1.0,"Operating Vehicles, Mechanized Devices, or Equ...",Operations Monitoring
295147,53-7081.00,Refuse and Recyclable Material Collectors,"Drive trucks, following established routes, th...",4.73,4.A.3.a.4.I01.D06,Operate vehicles or material-moving equipment.,1.0,"Operating Vehicles, Mechanized Devices, or Equ...",Operation and Control
295148,53-7121.00,"Tank Car, Truck, and Ship Loaders","Operate industrial trucks, tractors, loaders, ...",4.24,4.A.3.a.4.I01.D06,Operate vehicles or material-moving equipment.,1.0,"Operating Vehicles, Mechanized Devices, or Equ...",Operations Monitoring


In [23]:
skill_importance = all_tasks.groupby("skill")["task_importance"].sum().sort_values(ascending=False).reset_index().rename({"task_importance":"skill_importance_weighting_across_tasks"},axis=1)
skill_importance                                                                                               
                                                                                                       

,skill,skill_importance_weighting_across_tasks
0,Reading Comprehension,91748.41
1,Critical Thinking,89735.86
2,Active Listening,89192.47
3,Monitoring,82845.18
4,Writing,79904.26
5,Speaking,60982.38
6,Social Perceptiveness,55393.04
7,Service Orientation,53852.53
8,Coordination,53249.86
9,Active Learning,46098.19
